In [1]:
import requests
import json
import pandas as pd
from src.inference import *
pd.set_option('display.max_colwidth', None)

# Creating AWS Endpoint using Lambda with Serverless plugins

Serverless developers can use the AWS Serverless Application Model framework (AWS SAM). AWS SAM creates and manages serverless applications based on templates.

### Inital Setup - install AWS SAM globally:

Please run the following in your terminal:

``` brew tap aws/tap ```

``` brew install aws-sam-cli ```

### Create the service locally:

``` sam init ```

Follow the on-screen prompts:
- Select AWS Quick Start Templates as the template source
- Choose Image as the package type
- Select amazon/python3.9-base as the base image
- Enter application name: formality-api
- Select the desired ML framework: Scikit-learn Machine Learning Inference

Following the creation of the template, the following has been changed in the ./formality-api folder:
- Amened the yaml file to suit the application
- Amended the Dockerfile to point to the correct paths for inference code and saved models. Also added in a python command to download a nltk corpus the allow sentence tokenization
- Created /models folder and uploaded vectorizer and model .pkl
- Created api folder which includes api_inference code that is used to make formality predictions
- Amended app.py which uses the inference code and returns the sentences, formality scores and formality classification
- Added required dependencies to requirements.txt

### Test the local service:

Build the container: ``` sam build ```

Invoke the container locally and test using a mock event.json file: ``` sam local invoke InferenceFunction --event events/event.json ```

Run the container locally: ``` sam local start-api ```

### Once the local endpoint is created, we can test it using some python code:

In [2]:
url = "http://127.0.0.1:3000/formality-score/"

payload = "As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you’ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide. You’ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow."
headers = {'Content-Type': 'application/json'}

response = requests.request("POST", url, headers=headers, data=payload.encode('utf-8'))

print(response.text)

sentences = response.json()['sentences']
scores = response.json()['scores']
formality = response.json()['formality']

sentence_formality_df = pred_to_df(sentences, scores)
display(sentence_formality_df)

{"sentences": ["As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you\u2019ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide.", "You\u2019ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow."], "scores": [1.48, 1.62], "formality": ["Formal", "Formal"]}


,Sentence,Formality Score,Formality
0,"As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you’ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide.",1.48,Formal
1,"You’ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow.",1.62,Formal


### Deploy the working service to AWS Serverless Lambda:

Use the sam deployment command that guides you through the process:

```sam deploy --guided```

### Test the AWS endpoint using python requests:
#### Formal test:

In [3]:
f = open("./data/sanity_check/formal_text.txt", "r")
formal = f.read()
f.close()

url = "https://i9ie7dtl73.execute-api.eu-west-2.amazonaws.com/Prod/formality-score/"
payload = formal
response = requests.request("POST", url, headers=headers, data=payload.encode('utf-8'))

print(response.text)

sentences = response.json()['sentences']
scores = response.json()['scores']
formality = response.json()['formality']

sentence_formality_df = pred_to_df(sentences, scores)
display(sentence_formality_df)

{"sentences": ["As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you\u2019ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide.", "You\u2019ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow."], "scores": [1.48, 1.62], "formality": ["Formal", "Formal"]}


,Sentence,Formality Score,Formality
0,"As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you’ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide.",1.48,Formal
1,"You’ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow.",1.62,Formal


#### Informal test:

In [4]:
f = open("./data/sanity_check/informal_text.txt", "r")
informal = f.read()
f.close()

url = "https://i9ie7dtl73.execute-api.eu-west-2.amazonaws.com/Prod/formality-score/"
payload = formal
response = requests.request("POST", url, headers=headers, data=payload.encode('utf-8'))

print(response.text)

sentences = response.json()['sentences']
scores = response.json()['scores']
formality = response.json()['formality']

sentence_formality_df = pred_to_df(sentences, scores)
display(sentence_formality_df)

{"sentences": ["As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you\u2019ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide.", "You\u2019ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow."], "scores": [1.48, 1.62], "formality": ["Formal", "Formal"]}


,Sentence,Formality Score,Formality
0,"As a Trainee Artificial Intelligence (AI) Engineer in Kainos, you’ll be responsible for contributing to the development of high-quality solutions which integrate AI and ML technologies that delight our customers and impact the lives of users worldwide.",1.48,Formal
1,"You’ll do this whilst gaining awareness and learning about new AI technologies, frameworks and approaches, with talented colleagues that will help you to learn, develop and grow.",1.62,Formal


#### Job description test:

In [5]:
job_desc = "As a Lead Software Engineer (Consultant) in Kainos, you’ll be responsible for leading teams and developing high quality solutions which delight our customers and impact the lives of users worldwide. It’s a fast-paced environment so it is important for you to make sound, reasoned decisions. You’ll do this whilst learning about new technologies and approaches, with talented colleagues that will help you to learn, develop and grow. You’ll manage, coach and develop a small number of staff, with a focus on managing employee performance and assisting in their career development. You’ll also provide direction and leadership for your team as you solve challenging problems together.  As the technical leader in the team, you will also interact with customers, share knowledge and mentor those around you. Expertise in designing, building, testing and maintaining modern software applications. Technical leadership of teams building and testing modern, scalable, secure, performant applications in line with software development principles, practices and patterns. Experience of technical ownership for a product/software project, including architecture, estimation, product planning and user story/requirement creation. Expertise in software design and development across all layers of an application. We are passionate about developing people – a demonstrated ability in managing, coaching and developing junior members of your team and wider community. Experience with the latest Continuous Integration and Continuous Delivery techniques. Good communication skills, with the ability to communicate issues to technical and non-technical people. Active participation in knowledge sharing activities, both within the team and at a wider capability level and externally where appropriate. Experience of debugging and troubleshooting live applications. Experience of multiple programming languages and data storage technologies. Knowledge of public cloud platforms, such as AWS and Azure, including SaaS and PaaS offerings. Our vision is to enable outstanding people to create digital solutions that have a positive impact on people’s lives. Our values aren't abstract; they are the behaviours we expect from each other every day and underpin everything that we do. We expect everyone to display our values by being determined in how obstacles are overcome; honest when dealing with others; respectful of how you treat others; creative to find solutions to complex problems and cooperative by sharing information, knowledge and experience. These values, applied collectively, help to produce an outstanding Kainos person, team and culture. At Kainos we use technology to solve real problems for our customers, overcome big challenges for businesses, and make people’s lives easier. We build strong relationships with our customers and go beyond to change the way they work today and the impact they have tomorrow. Our two specialist practices, Digital Services and Workday, work globally for clients across healthcare, commercial and the public sector to make the world a little bit better, day by day. For more information, see kainos.com"

url = "https://i9ie7dtl73.execute-api.eu-west-2.amazonaws.com/Prod/formality-score/"
payload = job_desc
response = requests.request("POST", url, headers=headers, data=payload.encode('utf-8'))

print(response.text)

sentences = response.json()['sentences']
scores = response.json()['scores']
formality = response.json()['formality']

sentence_formality_df = pred_to_df(sentences, scores)
display(sentence_formality_df)

{"sentences": ["As a Lead Software Engineer (Consultant) in Kainos, you\u2019ll be responsible for leading teams and developing high quality solutions which delight our customers and impact the lives of users worldwide.", "It\u2019s a fast-paced environment so it is important for you to make sound, reasoned decisions.", "You\u2019ll do this whilst learning about new technologies and approaches, with talented colleagues that will help you to learn, develop and grow.", "You\u2019ll manage, coach and develop a small number of staff, with a focus on managing employee performance and assisting in their career development.", "You\u2019ll also provide direction and leadership for your team as you solve challenging problems together.", "As the technical leader in the team, you will also interact with customers, share knowledge and mentor those around you.", "Expertise in designing, building, testing and maintaining modern software applications.", "Technical leadership of teams building and tes

,Sentence,Formality Score,Formality
0,"As a Lead Software Engineer (Consultant) in Kainos, you’ll be responsible for leading teams and developing high quality solutions which delight our customers and impact the lives of users worldwide.",1.67,Formal
1,"It’s a fast-paced environment so it is important for you to make sound, reasoned decisions.",0.56,Formal
2,"You’ll do this whilst learning about new technologies and approaches, with talented colleagues that will help you to learn, develop and grow.",0.72,Formal
3,"You’ll manage, coach and develop a small number of staff, with a focus on managing employee performance and assisting in their career development.",2.07,Formal
4,You’ll also provide direction and leadership for your team as you solve challenging problems together.,-0.35,Informal
5,"As the technical leader in the team, you will also interact with customers, share knowledge and mentor those around you.",0.05,Formal
6,"Expertise in designing, building, testing and maintaining modern software applications.",0.54,Formal
7,"Technical leadership of teams building and testing modern, scalable, secure, performant applications in line with software development principles, practices and patterns.",3.10,Formal
8,"Experience of technical ownership for a product/software project, including architecture, estimation, product planning and user story/requirement creation.",2.49,Formal
9,Expertise in software design and development across all layers of an application.,0.70,Formal


### Test the AWS endpoint using Postman:

![image](./images/postman_test.png)